In [ ]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

pasrser = argparse.ArgumentParser(description = "This is a demo program")

device = "cuda" if torch.cuda.is_available() else "cpu"

# Hyper Parameters
block_size = 128
batch_size = 32
max_iters = 200
learning_rate = 3e-4
eval_iters = 100
n_embd = 384 # Num of attributes given to embedding
n_layers = 4
dropout = 0.2
n_head = 4

# Data Loader

In [ ]:
chars = ""
with open("openwebtext/vocab.txt", "r", encoding="utf-8") as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

# Tokenizer

In [ ]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

# Train and Val Split

In [ ]:
def get_random_chunk(split):
    filename = "openwebtext/train_split.txt" if split=="train" else "openwebtext/val_split.txt"
    with open(filename, "rb") as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size)-block_size*batch_size)
            # Seek to the random position and read block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)
            # Decode the block to a string, ignoring invalid byte sequences
            decoded_block = block.decode("utf-8", erros="ignore").replace("\r","")
            # Train and Test Splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

# Estimate Loss

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train","val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# PyTorch Language Model
<br> Decoder Only GPT Model

In [ ]:
class Head(nn.Module):
    """ One Head of Self Attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head_size)
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # Compute Attention Scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5  # (B,T,hs) @ (B,hs,T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
        wei = self.dropout(wei)
        # Perform weighted aggregation of the values
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ Multiple Heads of Self Attention in Parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj =  nn.Linear(head_size*num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Concatenate each head to get back the list of total features
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # Dropout makes certain percentage of neurons to dropout so as to prevent overfitiing
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ Simple Linear Layer followed by a Non Linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
                                    nn.Linear(n_embd, 4*n_embd),
                                    nn.ReLU(),
                                    nn.Linear(4*n_embd, n_embd),
                                    nn.Dropout(dropout),
                                )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    ''' Transformer Block '''
    def __init__(self, n_embd, n_head):
        # n_embd - Embedding Dimension (Number of features)
        # n_head - Number of heads we would like
        super().__init__()

        # Each head will capture head_size number of features
        head_size = n_embd//n_head
        # Self Attention
        self.sa = MultiHeadAttention(n_head, head_size)
        # Feed Forward
        self.ffwd = FeedForward(n_embd)
        # Two Layer Norms
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # Check Architecture of Decoder
        # MultiHead Self Attention -> Add and Norm --> Feed Fwd --> Add and Norm
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # Creates n_layer blocks sequentially i.e. 4 decoder layers
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layers)])

        # Final Normalization Layer
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self.__init__weights)

    def __init__weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, index, targets=None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

# PyTorch Optimizer

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter%eval_iters==0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)